### Laboratorium 7 Jak działa GPT?
#### Implemetacja transformera część 1

Imię i nazwisko: ..........................

Punktacja:
* 6 pkt. prawidłowa implementacja 
* 2 pkt. wnioski

Zaimplementujemy:
* model oparty o Embedding (to już było), ale przetwarzający sekwencje znaków o długości *seq_len*
    * ten model dalej korzysta tylko z ostatniego znaku
    * celem jest oswojenie się z kształtem danych na znanym modelu
* model posiadający jedną głowę atencji
* model o wielu głowach atencji

Model będziemy dalej rozwijać na kolejnym laboratorium. Kod z dzisiaj będzie używany jako punkt startowy i potrzebny za tydzień.

#### Źródła
* https://youtu.be/kCc8FmEb1nY?si=wYbFi5JB3x-R8375
* https://github.com/karpathy/nanoGPT
* https://arena-chapter1-transformer-interp.streamlit.app/

In [ ]:
import requests
import torch
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
import random

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# to jest do assertów, proszę zignorować
def set_seeds(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

#### Przygotowanie danych uczących

In [ ]:
# jako tekst ponownie wykorzystamy HPMOR rozdziały 1-10
# Eliezer Yudkowsky, Harry Potter and the Methods of Rationality https://hpmor.com/
url = "https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/data/hpmor_chapters_1-10.txt?raw=true"
response = requests.get(url)
text = response.text

In [ ]:
len(text)

In [ ]:
characters = sorted(set(text))
vocab_size = len(characters)
idx_to_ch = {i: c for i, c in enumerate(characters)}
ch_to_idx = {c: i for i, c in enumerate(characters)}

In [ ]:
# unikalne znaki w tekście
print(characters)

In [ ]:
print(vocab_size)

In [ ]:
def code(text):
    # zamienia tekst na listę tokenów (indeksów)
    return [ch_to_idx[c] for c in text]

In [ ]:
code("ala ma kota")

In [ ]:
def decode(tokens):
    # zamienia listę tokenów (indeksów) na tekst
    return ''.join(idx_to_ch[i] for i in tokens)

In [ ]:
decode([53, 64, 53, 1, 65, 53, 1, 63, 67, 72, 53])

In [ ]:
# kodujemy cały tekst (zamieniamy tekst na listę tokenów)
text_coded = code(text)
# tensor zawierający dane uczące
train = torch.tensor(text_coded)

In [ ]:
def get_batch(data, seq_len=8, batch_size=4):
    '''
    Funkcja zwraca batch danych.
    data (tensor) - dane uczące
    seq_len (int) - długość sekwencji
    batch_size (int) - rozmiar batcha

    X - tensor o kształcie (batch_size, seq_len)
    y - tensor o kształcie (batch_size, seq_len)
    '''
    n = len(data)
    starts = np.random.randint(0, n - seq_len, batch_size)
    X = torch.stack([data[s:s + seq_len] for s in starts])
    y = torch.stack([data[s + 1: s + seq_len + 1] for s in starts])
    return X.to(device), y.to(device)

In [ ]:
x, y = get_batch(train, batch_size=32)

In [ ]:
# tak wyglądają dane uczące
# y zawiera następny token dla każdego tokenu w X (nie tylko dla ostatniego tokenu sekwencji!)
# widzimy, że sekwencje w y są przesunięte o 1 względem sekwencji w X
x, y = get_batch(train)
print(x)
print(y)

In [ ]:
# zamieniamy tokeny na znaki i wypisujemy sekwencje x: ostatni token sekwencji y
for xi, yi in zip(x, y):
    print(decode(xi.tolist()), ": ", decode([yi.tolist()[-1]]))

#### Generacja tekstu

In [ ]:
def generate_text(start_seq, model, max_size, seq_len):
    '''
    Funkcja generuje tekst.
    start_seq (str) - początek tekstu, podany przez użytkownika
    model - sieć neuronowa
    max_size (int) - zadana długość tekstu
    seq_len (int) - długość sekwencji podawanej na wejście modelu
    '''
    for i in range(max_size):
        x = code(start_seq[-seq_len:]) #<- zmiana - ograniczamy dane wejściowe do ostatnich seq_len tokenów
        logits = model(torch.tensor([x], device=device)) #<- zmiana, x umieszczamy w [], bo jest to tensor o kształcie (1, seq_len)
        probs = F.softmax(logits, dim=-1) #<- zmiana dim=-1 - softmax jest obliczany po ostatnim wymiarze
        probs = probs[0, -1, :].cpu().detach().numpy() # <- zmiana, probs ma teraz kształt (batch_size, seq_len, vocab_size)
        # probs[0, -1, :] - wybieramy element batcha o indeksie 0 (jedyny)i sekwencji o indeksie -1 (ostatni token) 
        next_ch = idx_to_ch[np.random.choice(vocab_size, p=probs)]
        start_seq += next_ch
    return start_seq

### Model oparty tylko o warstwę embedding
* uwaga: to jest to samo co robiliśmy na lab 2 i lab 4, czyli przewidujemy znak tylko na podstawie poprzedniego znaku
* ale kształt danych wejściowych jest już dostosowany do wykorzystania większej liczby elementów sekwencji
* z innych elementów sekwencji korzystamy w kolejnych krokach


### Zaimplementować EmbeddingModel
* model ma jedną warstwę embedding
* warstwa embedding ma kształt (*vocab_size*, *vocab_size*)
* warstwa embedding jest warstwą typu nn.Embedding
* warstwa embedding zwraca logity o kształcie (*batch_size*, *seq_len*, *vocab_size*)



In [ ]:
class EmbeddingModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__() 
         #TODO

    def forward(self, X):
         #TODO
        return logits   #TODO

In [ ]:
model = EmbeddingModel(vocab_size).to(device)

In [ ]:
# wypisujemy kształty parametrów modelu
for name, param in model.named_parameters():
    print(name)
    print(param.shape)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f'Liczba parametrów modelu: {total_params}')

In [ ]:
# wyjście modelu ma kształt (*batch_size*, *seq_len*, *vocab_size*)
y_hat = model(x)
y_hat.shape

In [ ]:
batch_size = 256
seq_len = 8
n_steps = 1500
model = EmbeddingModel(vocab_size).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=5)

### Uzupełnić pętlę uczenia
* argumenty F.cross_entropy (logity i y) powinny mieć kształt (*batch_size* * *seq_len*, *vocab_size*)
* wskazówka: użyj .view do zmiany kształtu


In [ ]:
def train_loop(model, optimizer, n_steps, batch_size, seq_len, vocab_size):
    losses = []
    for step in range(n_steps):
        optimizer.zero_grad()
        x, y = get_batch(train, batch_size=batch_size)
        logits = model(x)
        loss = #TODO
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
        if step % 100 == 0:
            print(f"Krok {step}: Loss = {loss.item():.4f}")
    return losses

In [ ]:
losses = train_loop(model, optimizer, n_steps, batch_size, seq_len, vocab_size)

In [ ]:
plt.plot(losses);

In [ ]:
generate_text('T', model, 100, seq_len)

### Atencja o jednej głowie (Attention head)

Atencja o jednej głowie jest zdefiniowana następująco:

$$Q = X W_Q,\quad K = X W_K,\quad V = X W_V$$
$$\text{scores} = \frac{QK^{T}}{\sqrt{d_k}}$$
$$ \text{mask}_{ij} = 
\begin{cases}
1 & \text{if } j \leq i \\
0 & \text{if } j > i
\end{cases} \qquad \text{macierz trójkątna dolna}$$
$$\text{masked\_scores}_{ij} = \begin{cases}
\text{scores}_{ij} & \text{if } \text{mask}_{ij} = 1 \\
-\infty & \text{if } \text{mask}_{ij} = 0
\end{cases} $$
$$\text{attention} = \text{softmax}\left(\text{masked\_scores}\right)$$
$$ \text{weighted\_values} = \text{attention} \cdot V $$
$$ out = \text{weighted\_values} \cdot W_{out} $$

Implementujemy klasę **AttentionHead** (rysunek po lewej) oraz **AttentionHeadModel** (rysunek po prawej) według poniższego schematu:
<img src="https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/pic/attention_head.png?raw=true" alt="Atencja o jednej głowie" width="800" height="500">

### Zaimplementować klasę AttentionHead
* pomocny jest przykład z wykładu 7
* mnożenie przez macierze *Wk*, *Wq*, *Wv*, *Wout* jest realizowane przez warstwy nn.Linear
    * macierze *Wk*, *Wq*, *Wv* mają kształt (*d_model*, *d_head*)
    * macierz *Wout* ma kształt (*d_head*, *d_model*)
    * wykorzystujemy tylko macierze wag bez biasu (argument bias=False w konstruktorze nn.Linear)
* do wyznaczenia $K^T$ można użyć .transpose(-2, -1)
* wykorzystać funkcję torch.tril do tworzenia maski
* wykorzystać funkcję .masked_fill do maskowania wartości w macierzy *scores*


In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, d_model, d_head):
        super(AttentionHead, self).__init__()
        self.d_head = d_head
        self.Wk = #TODO
        self.Wq = #TODO
        self.Wv = #TODO
        self.Wout = #TODO
        

    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        K = #TODO    
        Q = #TODO
        V = #TODO
        scores = #TODO
        mask = #TODO
        masked_scores = #TODO
        attention = #TODO
        weighted_values = #TODO
        out = #TODO
        return out

In [ ]:
batch_size = 2
seq_len = 8
d_model = 16
d_head = 16 # teraz mamy jedną głowę, więc d_model = d_head

In [ ]:
set_seeds(42)
x = torch.randn(batch_size, seq_len, d_model, device=device)
x.shape

In [ ]:
model = AttentionHead(d_model, d_head).to(device)
out = model(x)
out.shape

In [ ]:
assert out.shape == torch.Size([2, 8, 16])

In [ ]:
assert torch.allclose(out[0, :, 0], torch.tensor([0.2119,  0.1579,  0.0388, -0.0663,  0.0123,  0.0133,  0.0630,  0.0521], device=device), atol=10**(-4))

### Zaimplementuj klasę AttentionHeadModel
(rysunek powyżej)
Model ten jest zbudowany z:
* warstwy embedding o kształcie (*vocab_size*, *d_model*)
* warstwy AttentionHead 
* warstwy liniowej out o kształcie (*d_model*, *vocab_size*) (typu nn.Linear, ta warstwa ma bias)


In [ ]:
class AttentionHeadModel(nn.Module):
    def __init__(self, vocab_size, d_head, d_model):
        super(AttentionHeadModel, self).__init__()
        self.emb = #TODO
        self.attention_head = #TODO
        self.linear_out = #TODO

    def forward(self, x):
        #TODO
        return logits

In [ ]:
set_seeds(37)
batch_size = 256
seq_len = 8
d_model = 16
d_head = 16
n_steps = 1500
model = AttentionHeadModel(vocab_size, d_head, d_model).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1.5)

In [ ]:
x = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
out = model(x)

In [ ]:
assert out.shape == torch.Size([256, 8, 79])

In [ ]:
assert torch.allclose(out[0, :, 0], torch.tensor([-0.0923, -0.0804,  0.0360,  0.0703,  0.0691,  0.0821,  0.0501,  0.0606], device=device), atol=10**(-4))

In [ ]:
for name, param in model.named_parameters():
    print(name)
    print(param.shape)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f'Liczba parametrów modelu: {total_params}')

In [ ]:
losses = train_loop(model, optimizer, n_steps, batch_size, seq_len, vocab_size)

In [ ]:
generate_text('T', model, 100, seq_len)

In [ ]:
plt.plot(losses);

### Atencja o wielu głowach (Multihead attention)

<div style="display: flex; justify-content: space-between;">
    <img src="https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/pic/multi_head_attention_simple.png?raw=true" alt="Struktura sieci" width="500">
    <img src="https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/pic/multihead.png?raw=true" alt="Atencja o wielu głowach" width="500">
</div>


### Zaimplementuj klasę MultiHeadAttention

(żółta część na rysunku powyżej)
* model ma wiele (*n_heads*) głów atencji
* każda głowa atencji jest typu AttentionHead
* wykorzystać nn.ModuleList do przechowywania głów atencji
* wyjścia wszystkich głów atencji są sumowane

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, d_head, d_model):
        super().__init__()
        self.attention_heads = #TODO

    def forward(self, x):
        #TODO
        return x

### Zaimplementuj klasę MultiHeadModel
* w odróżnieniu od AttentionHeadModel, MultiHeadModel ma wiele głów atencji
* model ten jest zbudowany z:
    * warstwy embedding o kształcie (*vocab_size*, *d_model*)
    * warstwy MultiHeadAttention <- **jedyna zmiana w porównaniu do AttentionHeadModel**
    * warstwy liniowej out o kształcie (*d_model*, *vocab_size*) (typu nn.Linear, ta warstwa ma bias)


In [ ]:
class MultiHeadModel(nn.Module):
    def __init__(self, n_heads, vocab_size, d_head, d_model):
        super().__init__()
        self.emb = #TODO
        self.linear_out = # TODO
        self.multi_head_attention = #TODO

    def forward(self, x):
        #TODO
        return logits

In [ ]:
set_seeds(28)
batch_size = 256
seq_len = 8
d_model = 16
d_head = 4
n_heads = 4
model = MultiHeadModel(n_heads, vocab_size, d_head, d_model).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1.5)

In [ ]:
x = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
out = model(x)

In [ ]:
assert out.shape == torch.Size([256, 8, 79])

In [ ]:
assert torch.allclose(out[0, :, 0], torch.tensor([0.2087, 0.0654, 0.0244, 0.0091, 0.0283, 0.0567, 0.0157, 0.0409], device=device), atol=10**(-4))

In [ ]:
for name, param in model.named_parameters():
    print(name)
    print(param.shape)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f'Liczba parametrów modelu: {total_params}')

In [ ]:
losses = train_loop(model, optimizer, n_steps, batch_size, seq_len, vocab_size)

In [ ]:
generate_text('T', model, 100, seq_len)

In [ ]:
plt.plot(losses);

### Wnioski
1. Porównaj tekst wygenerowany za pomocą EmbeddingModel, AttentionHeadModel i MultiHeadModel oraz wartości funkcji straty (loss) i liczby parametró modeli
3. Jak zwiększanie długości sekwencji wpływa na liczbę parametrów wszystkich modeli? Odpowiedź uzasadnij
4. Dlaczego MultiHeadModel ma tyle samo parametrów co AttentionHeadModel?